In [1]:
import pandas as pd

books = pd.read_csv("books_with_categories.csv")

In [2]:
from transformers import pipeline
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k = None,
                      device = "mps")
classifier("I love this!")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528684265911579},
  {'label': 'neutral', 'score': 0.005764597561210394},
  {'label': 'anger', 'score': 0.004419785924255848},
  {'label': 'sadness', 'score': 0.002092391485348344},
  {'label': 'disgust', 'score': 0.001611993182450533},
  {'label': 'fear', 'score': 0.0004138521908316761}]]

In [3]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.6548411250114441},
  {'label': 'neutral', 'score': 0.16985182464122772},
  {'label': 'sadness', 'score': 0.1164092868566513},
  {'label': 'surprise', 'score': 0.020700648427009583},
  {'label': 'disgust', 'score': 0.019100768491625786},
  {'label': 'joy', 'score': 0.015161257237195969},
  {'label': 'anger', 'score': 0.003935155458748341}]]

In [4]:
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296029329299927},
  {'label': 'neutral', 'score': 0.14038550853729248},
  {'label': 'fear', 'score': 0.06816217303276062},
  {'label': 'joy', 'score': 0.04794248566031456},
  {'label': 'anger', 'score': 0.009156341664493084},
  {'label': 'disgust', 'score': 0.0026284747291356325},
  {'label': 'sadness', 'score': 0.0021221621427685022}],
 [{'label': 'neutral', 'score': 0.449371874332428},
  {'label': 'disgust', 'score': 0.2735908627510071},
  {'label': 'joy', 'score': 0.10908269137144089},
  {'label': 'sadness', 'score': 0.09362723678350449},
  {'label': 'anger', 'score': 0.04047820717096329},
  {'label': 'surprise', 'score': 0.026970140635967255},
  {'label': 'fear', 'score': 0.006879061460494995}],
 [{'label': 'neutral', 'score': 0.6462149620056152},
  {'label': 'sadness', 'score': 0.24273452162742615},
  {'label': 'disgust', 'score': 0.04342262074351311},
  {'label': 'surprise', 'score': 0.028300486505031586},
  {'label': 'joy', 'score': 0.01421143

In [5]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [6]:
sorted(predictions[0], key=lambda x: x["label"])

[{'label': 'anger', 'score': 0.009156341664493084},
 {'label': 'disgust', 'score': 0.0026284747291356325},
 {'label': 'fear', 'score': 0.06816217303276062},
 {'label': 'joy', 'score': 0.04794248566031456},
 {'label': 'neutral', 'score': 0.14038550853729248},
 {'label': 'sadness', 'score': 0.0021221621427685022},
 {'label': 'surprise', 'score': 0.7296029329299927}]

In [7]:
import numpy as np

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [8]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [10]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 5197/5197 [12:58<00:00,  6.68it/s]


In [11]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [12]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn13
0,0.064134,0.273591,0.928169,0.932798,0.646215,0.967158,0.729603,9780002005883
1,0.612618,0.348286,0.942528,0.704421,0.887940,0.111690,0.252545,9780002261982
2,0.064134,0.104007,0.972321,0.767238,0.549477,0.111690,0.078765,9780006178736
3,0.351483,0.150723,0.360707,0.251881,0.732686,0.111690,0.078765,9780006280897
4,0.081412,0.184495,0.095043,0.040564,0.884389,0.475881,0.078765,9780006280934
...,...,...,...,...,...,...,...,...
5192,0.148209,0.030642,0.919165,0.255169,0.853722,0.980877,0.030656,9788172235222
5193,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765,9788173031014
5194,0.009997,0.009929,0.339217,0.947779,0.375756,0.066685,0.057625,9788179921623
5195,0.064134,0.104007,0.459269,0.759455,0.951104,0.368110,0.078765,9788185300535


In [13]:
books = pd.merge(books, emotions_df, on = "isbn13")

In [14]:
books.to_csv("books_with_emotions.csv", index = False)